In [ ]:
pip install requests
pip install lxml
pip install pandas
pip install openpyxl
pip install cobra
pip install cobrakbase
# for windows only, download the file from https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyeda firstly
# pip install pyeda-0.28.0-cp37-cp37m-win_amd64.whl  
pip install modelseedpy
pip install reframed


In [1]:
org_name = 'mko'
org_full_name = 'Methylomonas_koyamae'

In [2]:
org_name = 'mbry'
org_full_name = 'Methylocystis_bryophila'

In [3]:
from reframed import load_cbmodel
model = load_cbmodel(org_name+'_model\\'+org_name+'_add_from_modelseed_2.xml', flavor='fbc2')

c:\Users\vickenlee\AppData\Local\Programs\Python\Python39\lib\site-packages\reframed\io\sbml.py:169: UserWarning: Exchange reactions were not detected.
  warn("Exchange reactions were not detected.")


In [10]:
model.summary()

Metabolites:
c0 1017

Reactions:
enzymatic 0
transport 0
exchange 0
sink 0
other 920


# build model from modelseed

## Initialize new model

In [2]:
import cobra

new_model = cobra.Model(org_full_name)
new_model

Name,Methylocystis_bryophila
Memory address,1d4a8a0ff10
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


## KEGG ID mapping to modelseed

In [3]:
import pandas as pd

#org_name = 'mbry'
df_reaction = pd.read_excel(org_name+'_model\\'+org_name+'_clean_kegg_reaction.xlsx', index_col=1)  
titlename= ['entry_name','pathway_title','reaction_type','reaction_substrates','reaction_products']
df_keggID_modelseedID = pd.read_csv('reaction_keggID_modelseedID.txt', sep='\t')
mapping_reaction = pd.merge(df_reaction,df_keggID_modelseedID,how='left',on='Kegg_ID').drop_duplicates(subset=['Kegg_ID'], keep='first')[['ModelSeed_ID','Kegg_ID']+titlename]
mapping_reaction.to_excel(org_name+'_model\\'+org_name+'_mapping_reaction.xlsx', sheet_name='Sheet1', header=True)

print('done!')

done!


## add c0 reactions from ModelSeed Database

In [15]:
def add_ms_reaction(
    model,
    rxn_id,
    modelseed,
    compartment = 'c0',
    direction="forward",
):  # Xinli modified from modelseedpy.core.mseditorapi

    modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
    reaction_stoich = modelseed_reaction.cstoichiometry
    cobra_reaction = cobra.Reaction(rxn_id+'_'+ compartment)
    cobra_reaction.name = str(modelseed_reaction.data["name"])+'_'+compartment

    metabolites_to_add = {}
    for metabolite, stoich in reaction_stoich.items():
        id = metabolite[0]
        compound = modelseed.get_seed_compound(id).data
        if int(metabolite[1]) == 0:
            compartment_string = 'c0'
        elif int(metabolite[1]) == 1:
            compartment_string = 'e0'
        else:
            compartment_string = ''
            print(str(id)+' compartment wrong')

        metabolites_to_add[
            cobra.Metabolite(
                id+'_'+compartment_string, name=compound["name"]+'_'+compartment_string, compartment=compartment_string, 
                # formula = compound['formula']
            )
        ] = stoich
    cobra_reaction.add_metabolites(metabolites_to_add)
    cobra_reaction.reaction
    if direction == "reversible":
        cobra_reaction.lower_bound = -1000
    elif direction == "backward":
        cobra_reaction.lower_bound = -1000
        cobra_reaction.upper_bound = 0

    model.add_reactions([cobra_reaction])

In [ ]:
import cobra
import pandas as pd

mapping_reaction = pd.read_excel(org_name+'_model\\'+org_name+'_mapping_reaction.xlsx', index_col=0)  
modelseed_id_list = list(mapping_reaction['ModelSeed_ID'].dropna(axis = 0, how = 'any'))  # reaction ID that need to be added


import modelseedpy
modelseed_path = 'C:\\Users\\vickenlee\\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)

print('======================== \n %d reactions begin to added \n'%len(modelseed_id_list))

i,j = 0,0
for rxn_id in modelseed_id_list:
    reaction_type = mapping_reaction.query('ModelSeed_ID == "'+ rxn_id +'"')['reaction_type'].values[0]
    if reaction_type == 'reversible':
        direction = 'reversible'
    elif reaction_type == 'irreversible':
        direction = 'forward'
    try:
        add_ms_reaction(new_model, rxn_id = rxn_id, modelseed = modelseed, compartment = 'c0', direction = direction)
        print(rxn_id+' added')
        i+=1
    except:
        print(rxn_id+' added wrong !!!')
        j+=1

print('\n %d / %d reactions added successfully \n ====================='%(i,i+j))

for metabolite in new_model.metabolites:
    metaid = [meta for meta in metabolite.id.split('_') if meta.startswith('cpd')]
    if metaid:
        ms_metabolite = modelseed.get_seed_compound(metaid[0])
        if  ms_metabolite.formula == ms_metabolite.formula:
            metabolite.formula = ms_metabolite.formula
        metabolite.charge = ms_metabolite.data['charge']

cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_from_modelseed_1.xml')

In [20]:
for rxn in drift_model.reactions:
   rxn.annotation = {}
for met in drift_model.metabolites:
   met.annotation = {}

In [21]:
cobra.io.write_sbml_model(drift_model,filename=org_name+'_model\\'+org_name+'_add_from_modelseeddrf_1.xml')

#### check balance

In [5]:
def check_balance(reaction,H_metabolite=None):
    try:
        feedback = reaction.check_mass_balance()
        if feedback :
            print(reaction.id + ': ' + str(feedback))
            print('      '+ reaction.reaction)
            if H_metabolite:
                if 'H' in feedback and feedback.get('H') == feedback.get('charge'):
                    H_to_add={H_metabolite:-1*feedback.get('H')}
                    reaction.add_metabolites(H_to_add)
                    print('      H is added')
                    return(check_balance(reaction,H_metabolite))
            return int(0)
        else:
            #print(reaction.id + ': banlance')
            #print('      '+ reaction.reaction)
            return int(1)
    except:
        print(reaction.id + ': error')
        print('      '+ reaction.reaction)
        return int(0)



import cobra
import pandas as pd

H_metabolite = new_model.metabolites.get_by_id('cpd00067_c0')

i=0
for reaction in new_model.reactions:
    i=i+check_balance(reaction,H_metabolite)
print('After add H+, %d / %d reactions are balanced'%(i,len(new_model.reactions)))

cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_from_modelseed_2.xml')

rxn15989_c0: {'H': -2.0}
      cpd00363_c0 + 2.0 cpd19499_c0 <=> cpd00071_c0 + 2.0 cpd19500_c0
rxn05929_c0: {'C': 6.0, 'H': 10.0, 'O': 5.0}
      cpd00001_c0 --> cpd00108_c0
rxn07579_c0: {'charge': 2.0, 'C': -11.0, 'H': -21.0, 'N': -2.0, 'O': -7.0, 'P': -1.0, 'R': -1.0}
      cpd00004_c0 + cpd00067_c0 + cpd14940_c0 <=> cpd00003_c0 + cpd12458_c0
rxn07578_c0: {'charge': -2.0, 'C': 11.0, 'H': 21.0, 'O': 7.0, 'N': 2.0, 'P': 1.0}
      cpd14939_c0 <=> cpd00001_c0 + cpd14940_c0
rxn07576_c0: {'charge': -1.0}
      cpd00067_c0 + cpd11476_c0 + cpd11492_c0 --> cpd00011_c0 + cpd11493_c0 + cpd14938_c0
rxn07577_c0: {'charge': 2.0, 'C': -11.0, 'H': -21.0, 'N': -2.0, 'O': -7.0, 'P': -1.0}
      cpd00005_c0 + cpd00067_c0 + cpd14938_c0 <=> cpd00006_c0 + cpd14939_c0
rxn05350_c0: {'charge': -1.0}
      cpd00067_c0 + cpd11472_c0 + cpd11492_c0 --> cpd00011_c0 + cpd11490_c0 + cpd11493_c0
rxn05336_c0: {'charge': -3.0}
      cpd00006_c0 + 2.0 cpd00067_c0 + cpd11481_c0 <=> cpd00005_c0 + cpd11485_c0
rxn05323_c0

#### print unbalanced

In [6]:
for reaction in new_model.reactions:
    if check_balance(reaction)==0:
        print('           https://modelseed.org/solr/reactions/select?wt=json&q=id:'+reaction.id.replace('_c0',''))
        for meta, sto in reaction.metabolites.items():
            print('           '+meta.id+'    '+ str(meta.formula)+'    '+str(meta.charge))

        #new_model.metabolites.get_by_id('C00042_c0').charge


rxn15989_c0: {'H': -2.0}
      cpd00363_c0 + 2.0 cpd19499_c0 <=> cpd00071_c0 + 2.0 cpd19500_c0
           https://modelseed.org/solr/reactions/select?wt=json&q=id:rxn15989
           cpd19499_c0    None    0
           cpd19500_c0    None    0
           cpd00363_c0    C2H6O    0
           cpd00071_c0    C2H4O    0
rxn05929_c0: {'C': 6.0, 'H': 10.0, 'O': 5.0}
      cpd00001_c0 --> cpd00108_c0
           https://modelseed.org/solr/reactions/select?wt=json&q=id:rxn05929
           cpd00108_c0    C6H12O6    0
           cpd00001_c0    H2O    0
rxn07579_c0: {'charge': 2.0, 'C': -11.0, 'H': -21.0, 'N': -2.0, 'O': -7.0, 'P': -1.0, 'R': -1.0}
      cpd00004_c0 + cpd00067_c0 + cpd14940_c0 <=> cpd00003_c0 + cpd12458_c0
           https://modelseed.org/solr/reactions/select?wt=json&q=id:rxn07579
           cpd14940_c0    C29H54N2O8PRS    -2
           cpd12458_c0    C18H35OS    0
           cpd00004_c0    C21H27N7O14P2    -2
           cpd00067_c0    H    1
           cpd00003_c0    C21H26N7O14

In [22]:
new_model

Name,Methylocystis_bryophila
Memory address,1d4ea605910
Number of metabolites,1037
Number of reactions,921
Number of genes,0
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,c0


## add bio reaction from Drift model

In [3]:
import cobra

#drift_model_path = org_name+'_model\\ModelSeed_model_DSMZ_21852.xml'
drift_model_path = org_name+'_model\\ModelSeed_model_'+org_full_name+'.xml'
drift_model = cobra.io.read_sbml_model(drift_model_path)

In [4]:
import cobra
mpar_model = cobra.io.read_sbml_model('other_model\\Methylocystis_parvus.xml')
mpar_model


'M_3-oxooctadecanoylacp' is not a valid SBML 'SId'.
'M_R-3-hydroxyoctadecanoylacp' is not a valid SBML 'SId'.
'M_3-hydroxyvalerylcoa' is not a valid SBML 'SId'.
'M_Methylsuccinyl-CoA' is not a valid SBML 'SId'.
'M_Ethylmalonyl-CoA' is not a valid SBML 'SId'.
'R_valerate1.3.99' is not a valid SBML 'SId'.
'R_valerate4.2.1.17' is not a valid SBML 'SId'.
'R_valerate1.1.1.35' is not a valid SBML 'SId'.
'R_valerate2.3.1.16' is not a valid SBML 'SId'.
'R_valerate6.2.1.1' is not a valid SBML 'SId'.
'R_rxn5.4.99.63_c0' is not a valid SBML 'SId'.
'R_rxn1.1.99.2_c0' is not a valid SBML 'SId'.
'R_rxn1.3.1.85_c0' is not a valid SBML 'SId'.
'R_rxn5.1.99.1_c0' is not a valid SBML 'SId'.
'R_rxn5.4.99.2_c0' is not a valid SBML 'SId'.
'R_acyl-CoA_thioestearase' is not a valid SBML 'SId'.
'R_rxn2.7.1.165_c0' is not a valid SBML 'SId'.


Name,Methylocystis_parvus
Memory address,2df9b338280
Number of metabolites,1399
Number of reactions,1324
Number of genes,625
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol_0, Extracellular_0,"


In [7]:
new_model = cobra.io.read_sbml_model(org_name+'_model\\'+org_name+'_add_from_modelseed_2.xml')

No objective coefficients in model. Unclear what should be optimized


In [ ]:
maprlist = [rxn.id for rxn in mpar_model.reactions]
driftlist = [rxn.id for rxn in drift_model.reactions]
modellist = [rxn.id for rxn in new_model.reactions]

In [14]:
for rxn in maprlist:
    if not rxn in driftlist:
        print(rxn)

rxn08180_c0
rxn00973_c0
rxn03239_c0
rxn11962_c0
rxn04271_c0
rxn10118_c0
rxn05392_c0
rxn03046_c0
rxn00519_c0
rxn00258_c0
rxn00730_c0
rxn05367_c0
rxn03456_c0
rxn03360_c0
rxn02898_c0
rxn01547_c0
rxn04996_c0
rxn05404_c0
rxn02796_c0
rxn04048_c0
rxn01486_c0
rxn10205_c0
rxn05400_c0
rxn02090_c0
rxn01750_c0
rxn05814_c0
rxn00678_c0
rxn01331_c0
rxn05453_c0
rxn03142_c0
rxn02331_c0
rxn06493_c0
rxn10214_c0
rxn07680_c0
rxn08669_c0
rxn00658_c0
rxn02961_c0
rxn08971_c0
rxn01635_c0
rxn03887_c0
rxn03906_c0
rxn05358_c0
rxn03436_c0
rxn08976_c0
rxn05408_c0
rxn05293_c0
rxn02687_c0
rxn05514_c0
rxn00695_c0
rxn07670_c0
rxn05116_c0
rxn00655_c0
rxn01492_c0
rxn08118_c0
rxn05413_c0
rxn02703_c0
rxn06194_c0
rxn00259_c0
rxn10125_c0
rxn07875_c0
rxn02261_c0
rxn00465_c0
rxn05127_c0
rxn00305_c0
rxn07312_c0
rxn01701_c0
rxn05417_c0
rxn05049_c0
rxn01157_c0
rxn00758_c0
rxn00471_c0
rxn03848_c0
rxn00798_c0
rxn10213_c0
rxn05229_c0
rxn07721_c0
rxn02821_c0
rxn04068_c0
rxn09997_c0
rxn03208_c0
rxn00707_c0
rxn03095_c0
rxn00566_c0
rxn0

In [13]:
for rxn in maprlist:
    if not rxn in modellist:
        print(rxn)

rxn08180_c0
rxn05561_c0
rxn05250_c0
rxn05440_c0
rxn05625_c0
rxn00973_c0
rxn05457_c0
rxn04271_c0
rxn13783_c0
rxn10118_c0
rxn05392_c0
rxn03433_c0
rxn10231_c0
rxn12637_c0
rxn03046_c0
rxn09240_c0
rxn00519_c0
rxn00258_c0
rxn00730_c0
rxn05367_c0
rxn03456_c0
rxn02898_c0
rxn05654_c0
rxn08807_c0
rxn10344_c0
rxn01547_c0
rxn04996_c0
rxn05404_c0
rxn02796_c0
rxn10262_c0
rxn10126_c0
rxn10225_c0
rxn01486_c0
rxn00898_c0
rxn05444_c0
rxn10205_c0
rxn05400_c0
rxn02090_c0
rxn08556_c0
rxn01750_c0
rxn10228_c0
rxn05814_c0
rxn00678_c0
rxn00001_c0
rxn01331_c0
rxn11268_c0
rxn08817_c0
rxn00910_c0
rxn01297_c0
rxn05459_c0
rxn05453_c0
rxn08796_c0
rxn03142_c0
rxn02331_c0
rxn10214_c0
rxn12846_c0
rxn08308_c0
rxn08823_c0
rxn05434_c0
rxn07680_c0
rxn03248_c0
rxn05516_c0
rxn05733_c0
rxn09206_c0
rxn09209_c0
rxn12512_c0
rxn08669_c0
rxn00658_c0
rxn06672_c0
rxn02961_c0
rxn12641_c0
rxn00068_c0
rxn08971_c0
rxn01635_c0
rxn03887_c0
rxn09198_c0
rxn08838_c0
rxn03906_c0
rxn05358_c0
rxn02374_c0
rxn05437_c0
rxn08976_c0
rxn05319_c0
rxn0

In [2]:
def add_reaction_from_driftModel(
    new_model,
    drift_model,
    rxn_id,
): 
    reaction_drift = drift_model.reactions.get_by_id(rxn_id)
    cobra_reaction = cobra.Reaction(rxn_id)
    cobra_reaction.name = reaction_drift.name
    cobra_reaction.lower_bound = reaction_drift.lower_bound
    cobra_reaction.upper_bound = reaction_drift.upper_bound
    
    metabolites_to_add = {}
    for metabolite, stoich in reaction_drift.metabolites.items():
        id = metabolite.id
        name = metabolite.name
        charge = metabolite.charge
        compartment = metabolite.compartment
        
        if  metabolite.formula == metabolite.formula:
            formula = metabolite.formula
        else:
            formula = ''
        metabolites_to_add[
                cobra.Metabolite(id, name=name, compartment=compartment, formula=formula, charge=charge)
            ] = stoich

    
    cobra_reaction.add_metabolites(metabolites_to_add)
    new_model.add_reactions([cobra_reaction])

In [5]:
new_model = cobra.io.read_sbml_model(org_name+'_model\\'+org_name+'_add_from_modelseed_2.xml')
add_reaction_from_driftModel(new_model,drift_model,'bio1')
new_model.objective = new_model.problem.Objective(new_model.reactions.get_by_id('bio1').flux_expression, direction='max')

cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_from_modelseed_3_2.xml')

No objective coefficients in model. Unclear what should be optimized


## add trans reactions from drift model

In [6]:
drift_model.boundary

[<Reaction rxn13783_c0 at 0x1797fb80280>,
 <Reaction rxn13784_c0 at 0x1797fb79c70>,
 <Reaction rxn13782_c0 at 0x1797fdabaf0>,
 <Reaction EX_cpd00067_e0 at 0x17905080070>,
 <Reaction EX_cpd00106_e0 at 0x17905080f10>,
 <Reaction EX_cpd00075_e0 at 0x1790508a4f0>,
 <Reaction EX_cpd00036_e0 at 0x1790508afd0>,
 <Reaction EX_cpd04097_e0 at 0x1790508a460>,
 <Reaction EX_cpd11585_e0 at 0x1790508afa0>,
 <Reaction EX_cpd11582_e0 at 0x17905098850>,
 <Reaction EX_cpd00012_e0 at 0x17905098910>,
 <Reaction EX_cpd11586_e0 at 0x179050989d0>,
 <Reaction EX_cpd01012_e0 at 0x17905098a90>,
 <Reaction EX_cpd00001_e0 at 0x17905098b50>,
 <Reaction EX_cpd00063_e0 at 0x17905098c10>,
 <Reaction EX_cpd00058_e0 at 0x17905098cd0>,
 <Reaction EX_cpd00107_e0 at 0x17905098d90>,
 <Reaction EX_cpd01017_e0 at 0x17905098e50>,
 <Reaction EX_cpd00013_e0 at 0x17905098f10>,
 <Reaction EX_cpd11578_e0 at 0x17905098fd0>,
 <Reaction EX_cpd11591_e0 at 0x17905098ee0>,
 <Reaction EX_cpd00009_e0 at 0x179050986d0>,
 <Reaction EX_cpd00

In [7]:
#new_model = cobra.io.read_sbml_model(org_name+'_model\\'+org_name+'_add_from_modelseed_3.xml')

trans_reactions = []
ex_reactions = []
for reaction in drift_model.reactions:
    if reaction.id.startswith('EX_'):
        ex_reactions.append(reaction.id)
        continue
    for meta, sto in reaction.metabolites.items():
        if meta.id.endswith('_e0'):
            print(reaction.id)
            panding = input('add '+reaction.id+'? (1/0)')
            if panding == '1':
                trans_reactions.append(reaction.id)
            break


for rxnid in trans_reactions:
    add_reaction_from_driftModel(new_model,drift_model,rxnid)
    print(rxnid+' added')


boundary_reactions = [reaction.id for reaction in drift_model.boundary]
for rxnid in boundary_reactions:
    add_reaction_from_driftModel(new_model,drift_model,rxnid)
    print(rxnid+' added')

cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_from_modelseed_4_2.xml')

rxn05561_c0
rxn05625_c0
rxn10118_c0
rxn05654_c0
rxn10344_c0
rxn11268_c0
rxn05516_c0
rxn05319_c0
rxn05514_c0
rxn10481_c0
rxn10125_c0
rxn05466_c0
rxn05585_c0
rxn05145_c0
rxn05150_c0
rxn05552_c0
rxn05172_c0
rxn05175_c0
rxn05163_c0
rxn10122_c0
rxn05152_c0
rxn10571_c0
rxn10168_c0
rxn10042_c0
rxn05146_c0
rxn05594_c0
rxn05512_c0
rxn05528_c0
rxn05312_c0
rxn05153_c0
rxn05195_c0
rxn05209_c0
rxn10121_c0
rxn05618_c0
rxn10474_c0
rxn10343_c0
rxn05599_c0
rxn05627_c0
rxn05620_c0
rxn05656_c0
rxn05616_c0
rxn10043_c0
rxn05176_c0
rxn05179_c0
rxn05468_c0
rxn05621_c0
rxn05206_c0
rxn05605_c0
rxn05467_c0
rxn05255_c0
rxn10473_c0
rxn10163_c0
rxn10145_c0
rxn13160_c0
rxn05722_c0
rxn10161_c0
rxn12664_c0
rxn10113_c0
MeDH
MMO1


In [40]:
new_model.reactions.get_by_id('rxn05468_c0').upper_bound


1000.0

In [47]:
cobra_reaction = cobra.Reaction('rxn13109_c0')
cobra_reaction.name = 'Transport of methane'
cobra_reaction.lower_bound = 0
cobra_reaction.upper_bound = 1000
metabolites_to_add = {}
metabolites_to_add[cobra.Metabolite('cpd01024_e0', name='Methane_e0', compartment='e0', formula='CH4', charge=0)] = -1
metabolites_to_add[new_model.metabolites.get_by_id('cpd01024_c0')] = 1
cobra_reaction.add_metabolites(metabolites_to_add)
new_model.add_reactions([cobra_reaction])

new_model.add_boundary(new_model.metabolites.get_by_id("cpd01024_e0"), type="exchange")
new_model.reactions.get_by_id('EX_cpd01024_e0').annotation={}

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


Reaction identifier,EX_cpd01024_e0
Name,Methane_e0 exchange
Memory address,0x1d4ebf504f0
Stoichiometry,cpd01024_e0 <=> Methane_e0 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [52]:
cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_from_modelseed_4.xml')

In [59]:
new_model

Name,Methylocystis_bryophila
Memory address,1d4ebd78460
Number of metabolites,1142
Number of reactions,1087
Number of genes,0
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"c0, e0,"


## export to excel

In [1]:
# conda install <libsbml>
# pip install python-libsbml
import libsbml

r = libsbml.SBMLReader()
'''
doc = r.readSBML(
    ".\\5GB1_base_fermentation.xml"
)
'''
doc = r.readSBML(
    ".\\mbry_model\\mbry_add_bio_from_modelseed.xml"
)

m = doc.getModel()

for n in range(0, m.getNumReactions()):
    r = m.getReaction(n)
    # print reaction id
    print(
        #"Reaction ",
        r.getId(),
        ": ",
        end=""
    )

    # look at reactants
    numReactants = r.getNumReactants()
    if (numReactants > 1):
        s = m.getSpecies(r.getReactant(0).getSpecies())
        print(
            r.getReactant(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )
        for k in range(1, numReactants):
            # get species referred to by the reaction
            s = m.getSpecies(r.getReactant(k).getSpecies())
            print(
                "+ ",
                r.getReactant(k).getStoichiometry(),
                " ",
                s.getName(),
                " ",
                end=""
            )
    elif (numReactants == 1):
        # get species referred to by the reaction
        s = m.getSpecies(r.getReactant(0).getSpecies())
        print(
            r.getReactant(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )

    if (r.getReversible() == True):
        print("<=> ", end="")
    else:
        print("=> ", end="")

# look at products
    numProducts = r.getNumProducts()
    if (numProducts > 1):
        s = m.getSpecies(r.getProduct(0).getSpecies())
        print(
            r.getProduct(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )
        for k in range(1, numProducts):
            # get species referred to by the reaction
            s = m.getSpecies(r.getProduct(k).getSpecies())
            print(
                "+ ",
                r.getProduct(k).getStoichiometry(),
                " ",
                s.getName(),
                " ",
                end=""
            )
    elif (numProducts == 1):
        # get species referred to by the reaction
        s = m.getSpecies(r.getProduct(0).getSpecies())
        print(
            r.getProduct(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )
    print("\n")


R_rxn15116_c0 : 1.0   beta-D-Fructose 1,6-bisphosphate_c0  <=> 1.0   Glycerone-phosphate_c0  +  1.0   Glyceraldehyde3-phosphate_c0  

R_rxn00506_c0 : 1.0   H2O_c0  +  1.0   NAD_c0  +  1.0   Acetaldehyde_c0  <=> 1.0   NADH_c0  +  1.0   Acetate_c0  +  2.0   H+_c0  

R_rxn00536_c0 : 1.0   NADP_c0  +  1.0   Ethanol_c0  <=> 1.0   NADPH_c0  +  1.0   H+_c0  +  1.0   Acetaldehyde_c0  

R_rxn00543_c0 : 1.0   NAD_c0  +  1.0   Ethanol_c0  <=> 1.0   NADH_c0  +  1.0   H+_c0  +  1.0   Acetaldehyde_c0  

R_rxn00011_c0 : 1.0   CO2_c0  +  1.0   2-Hydroxyethyl-ThPP_c0  => 1.0   Pyruvate_c0  +  1.0   TPP_c0  +  1.0   H+_c0  

R_rxn02342_c0 : 1.0   Lipoamide_c0  +  1.0   2-Hydroxyethyl-ThPP_c0  => 1.0   S-Acetyldihydrolipoamide_c0  +  1.0   TPP_c0  

R_rxn01871_c0 : 1.0   Acetyl-CoA_c0  +  1.0   Dihydrolipoamide_c0  <=> 1.0   CoA_c0  +  1.0   S-Acetyldihydrolipoamide_c0  

R_rxn00148_c0 : 1.0   ATP_c0  +  1.0   Pyruvate_c0  => 1.0   ADP_c0  +  1.0   Phosphoenolpyruvate_c0  +  1.0   H+_c0  

R_rxn00459_c0 

## add reactions and metabolites note

https://narrative.kbase.us/#catalog/apps/kb_uploadmethods/import_file_as_fba_model_from_staging

In [31]:
import cobra
model = cobra.io.read_sbml_model(org_name+'_model\\'+org_name+'_add_from_modelseed_4.xml_model.xml')
model

'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


Name,
Memory address,1d4ec6ad220
Number of metabolites,1183
Number of reactions,1149
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"c0, e0"


In [33]:
rxnids = [rxn.id for rxn in model.reactions]

for rxn in new_model.reactions:
    if not rxn.id in rxnids:
        print(rxn.id)

rxn13783_c0
rxn13784_c0
rxn13782_c0


In [ ]:
# add gene
print('======================== \n genes of %d reactions begin to added \n'%i)
#mapping_reaction = pd.read_excel(org_name+'_model\\'+org_name+'_mapping_reaction.xlsx', index_col=0)  
i=1
for reaction in new_model.reactions:
    rxnid = [rxn for rxn in reaction.id.split('_') if rxn.startswith('rxn')]
    if rxnid:
        gene_list_str = '( '+mapping_reaction.query('ModelSeed_ID == "'+ rxnid[0] +'"')['entry_name'].values[0].replace(' ', ' or ').replace('mbry:', '')+' )'
        reaction.gene_reaction_rule = gene_list_str
    if i%100 == 0:
        print('%d / %d reactions genes added succesfully'%(i,len(new_model.reactions)))
    i+=1
print('%d / %d reactions genes added succesfully'%(len(new_model.reactions),len(new_model.reactions)))
